<a href="https://colab.research.google.com/github/Mergim01/RiverStreamClassification/blob/main/Notebooks/first_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install git+https://github.com/online-ml/river --upgrade

  Cloning https://github.com/online-ml/river to /tmp/pip-req-build-fdlfek1e
  Running command git clone -q https://github.com/online-ml/river /tmp/pip-req-build-fdlfek1e
  Created wheel for river: filename=river-0.10.1-cp37-cp37m-linux_x86_64.whl size=1658777 sha256=92428f7267f9f1597648ce92c4bbdb6bf09480584390afb121bc0a2375b1a961
  Stored in directory: /tmp/pip-ephem-wheel-cache-f2h7_h4w/wheels/a1/81/b6/35dc3f68cb6c44cc3967a6a2f40410150d6ed5df2ff2b83f41
Successfully built river


In [4]:
import pandas as pd
import river
import numpy as np
from tqdm.notebook import tqdm

In [7]:
df = pd.read_csv("https://raw.githubusercontent.com/Mergim01/RiverStreamClassification/main/Data/ClimateGPT.csv")
# Convert to Format
df.to_dict()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40999 entries, 0 to 40998
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  40999 non-null  object 
 1   id          40997 non-null  object 
 2   timestamp   40997 non-null  object 
 3   text        40992 non-null  object 
 4   user        40995 non-null  object 
 5   artificial  40995 non-null  float64
dtypes: float64(1), object(5)
memory usage: 1.9+ MB


In [8]:
df = df[["text","artificial"]]
df = df.dropna() #let us first remove all rows with NaNs
df[["artificial"]] = df[["artificial"]].applymap(np.int64)

from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)
#mergim ist schlau

In [9]:
# Convert to Tuple
train_set = train.to_records(index=False)
test_set = test.to_records(index=False)
train_set

rec.array([('Climate change: UN emissions gap report a thundering wake-up call https://t.co/O8Z4uD6jR9 #news #topstories #berkleybearnews', 1),
           ('Most of the time, I like it, but sometimes, I hate it.', 1),
           ('@8f0385bc383025ea9341 @d3c4b75fa31bc2f18c84 @e925e6092874aa51cefc Eliminating our reliance on massive shipping containers would be make a big dent on pollution. Our only major export is natural resource extraction which employs 2% of the population. All our eggs are in one basket and the cocks hold us ransom.', 0),
           ...,
           ('Climate change is the biggest threat we face. And this young college student’s plan to help...is inspiring others to do what we all can do today: take action.', 1),
           ('The mayor became a target after\xa0Imperial Beach filed a lawsuit\xa0against ExxonMobil, Chevron, BP and more than 30 other fossil-fuel companies demanding they pay the huge costs of defending the city from rising seas caused by the climate cris

In [10]:
from river.naive_bayes import MultinomialNB
from river.neighbors import KNNClassifier
from river.tree import HoeffdingTreeClassifier, ExtremelyFastDecisionTreeClassifier
from river import linear_model
from river.feature_extraction import BagOfWords,TFIDF
from river import preprocessing
from river.compose import Pipeline 
from river import optim

In [11]:
# Build Pipeline
pipe_nb = Pipeline(('vectorizer',BagOfWords(lowercase=True)),('nb',MultinomialNB()))
pipe_nb

Pipeline (
  BagOfWords (
    on=None
    strip_accents=True
    lowercase=True
    preprocessor=None
    tokenizer=<built-in method findall of re.Pattern object at 0x7f2afb5b7b90>
    ngram_range=(1, 1)
  ),
  MultinomialNB (
    alpha=1.
  )
)

In [12]:
# Specify Steps
pipe_nb.steps

OrderedDict([('vectorizer', BagOfWords (
                on=None
                strip_accents=True
                lowercase=True
                preprocessor=None
                tokenizer=<built-in method findall of re.Pattern object at 0x7f2afb5b7b90>
                ngram_range=(1, 1)
              )), ('nb', MultinomialNB (
                alpha=1.
              ))])

In [13]:
# Train
for text,label in tqdm(train_set):
    pipe_nb = pipe_nb.learn_one(text,label)

  0%|          | 0/32792 [00:00<?, ?it/s]

In [14]:
# Make a Prediction
test = 'Mergim ist keine Maschine'
pred = pipe_nb.predict_one(test)
# Prediction Prob
proba_ = pipe_nb.predict_proba_one(test)
print('Predicted test sentence as',pred,' and probability of classes for test as',proba_)

Predicted test sentence as 1  and probability of classes for test as {1: 0.905074262464416, 0: 0.09492573753558389}


In [ ]:
test_set[0][0]

'Prince William’s Earthshot Award honors Costa Rica and Milan in Climate Fight –\xa0CityNewss https://t.co/KZCLQGdchM'

In [ ]:
example = test_set[0]
pred = pipe_nb.predict_one(example[0])
# Prediction Prob
proba_ = pipe_nb.predict_proba_one(example[0])
print('sentence:',example[0])
print('Predicted test sentence as',pred,' and probability of classes for test as',proba_)
print('True Label:',example[1])

sentence: Prince William’s Earthshot Award honors Costa Rica and Milan in Climate Fight – CityNewss https://t.co/KZCLQGdchM
Predicted test sentence as 0  and probability of classes for test as {1: 5.988650917124152e-05, 0: 0.9999401134908233}
True Label: 0


In [ ]:
# Update the Model on the test data & Check Accuracy
metric = river.metrics.F1()
metrics = []
for text,label in tqdm(test_set):
    y_pred_before = pipe_nb.predict_one(text)
    metric = metric.update(label,y_pred_before)
    metrics.append(metric)
    # Has already learnt the pattern
    #pipe_nb = pipe_nb.learn_one(text,label)
print(metric)

  0%|          | 0/8199 [00:00<?, ?it/s]

F1: 73.41%


In [ ]:
metrics

[F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,
 F1: 73.41%,